In [1]:
# encoding=utf8
import tweepy
import csv
import re
import os
import json
import pandas as pd
import datetime

In [2]:
def dtparser (x): 
    try:
        return pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
    except ValueError:
        return pd.datetime.strptime(x, '%Y-%m-%d %H:%M')

In [3]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

In [4]:
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret) 
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=False)


In [5]:
in_file = "tweets09"

In [6]:
data = pd.read_csv("../input-Data/"+in_file)

In [7]:
# data.head()

In [8]:
# tweet = api.get_status(data.iloc[2][0])
# (datetime.datetime.now() - tweet.created_at).days
# date_time_str = '2019-12-8 13:00:00.243860'
# date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')
# ( datetime.datetime.now() - date_time_obj ).days

In [9]:
df = pd.DataFrame(columns=['event_id','tweet_id', 'user_id', 'screen_name', 'signup_time', 'followers_count', 'friends_count', 'location', 'verified', 
 'user_description', 'listed_count', 'favourites_count', 'statuses_count', 'default_profile', 'listed_count',
 'tweet_created_at', 'tweet_content_text', 'retweet_count', 'in_reply_to_status_id_str', 'in_reply_to_user_id',
 'favorite_count', 'entities'])
df2 = pd.DataFrame(columns=['event_id','tweet_id', 'user_id', 'screen_name', 'signup_time', 'followers_count', 'friends_count', 'location', 'verified', 
 'user_description', 'listed_count', 'favourites_count', 'statuses_count', 'default_profile', 'listed_count',
 'tweet_created_at', 'tweet_content_text', 'retweet_count', 'in_reply_to_status_id_str', 'in_reply_to_user_id',
 'favorite_count', 'entities'])

In [10]:
# df = df.append({'event_id':"row[1]",'tweet_id': "row[0]", 'user_id': tweet.user.id_str, 'screen_name': tweet.user.screen_name, 'signup_time': tweet.user.created_at, 'followers_count':tweet.user.followers_count,
#      'friends_count': tweet.user.friends_count, 'location':tweet.user.location, 'verified' :tweet.user.verified, 
#      'user_description':tweet.user.description.encode("utf-8"), 'listed_count': tweet.user.listed_count, 'favourites_count':tweet.user.favourites_count,
#      'statuses_count': tweet.user.statuses_count, 'default_profile':tweet.user.default_profile, 'listed_count': tweet.user.listed_count,
#      'tweet_created_at':tweet.created_at, 'tweet_content_text':tweet.text.encode("utf-8"), 'retweet_count':tweet.retweet_count,
#      'in_reply_to_status_id_str':tweet.in_reply_to_status_id_str, 'in_reply_to_user_id':tweet.in_reply_to_user_id,
#      'favorite_count':tweet.favorite_count, 'entities':tweet.entities}, ignore_index=True)

In [11]:
# df

In [12]:
get_last = True
date = None
current = 829

In [13]:
for index, row in data.iterrows():
    print "1: current={} and row ={} {}".format(current,row[1],datetime.datetime.now())
    if(row[1]>current):
        current =row[1]
        df.to_csv("output/hour10-events_{}.csv".format(in_file),encoding='utf-8')
        df.to_csv("output/backup/hour10-{}_{}.csv".format(index,in_file),encoding='utf-8')
        get_last=True
    if current == row[1] :
        if get_last :
            get_last = False
            for i in range(index,(index+1000)):
                try:
                    print "2: origin time={} id={} {}".format(date,row[0],datetime.datetime.now())
                    tweet = api.get_status(data.iloc[i][0])
                    date = tweet.created_at 
                    current = row[1]
                    break;
                except Exception , e:
                    print(e)
                    continue
            
            for i in range(0,1000):
                try:
                    tweet = api.get_status(data.iloc[index -i][0])
                    print "3: last time={} and id ={} {}".format(tweet.created_at,row[0],datetime.datetime.now())
                    df2 = df2.append({'event_id': row[1],'tweet_id': row[0], 'user_id': tweet.user.id_str, 'screen_name': tweet.user.screen_name.encode("utf-8").strip(), 'signup_time': tweet.user.created_at, 'followers_count':tweet.user.followers_count,
                 'friends_count': tweet.user.friends_count, 'location':tweet.user.location, 'verified' :tweet.user.verified, 
                 'user_description':tweet.user.description.encode("utf-8").strip(), 'listed_count': tweet.user.listed_count, 'favourites_count':tweet.user.favourites_count,
                 'statuses_count': tweet.user.statuses_count, 'default_profile':tweet.user.default_profile, 'listed_count': tweet.user.listed_count,
                 'tweet_created_at':tweet.created_at, 'tweet_content_text':tweet.text.encode("utf-8").strip(), 'retweet_count':tweet.retweet_count,
                 'in_reply_to_status_id_str':tweet.in_reply_to_status_id_str, 'in_reply_to_user_id':tweet.in_reply_to_user_id,
                 'favorite_count':tweet.favorite_count, 'entities':tweet.entities}, ignore_index=True)
                    df2.to_csv("output/last/lasttime_{}.csv".format(in_file),encoding='utf-8')
                    break
                except Exception , e:
                    print(e)
                    continue
        try:
            tweet = api.get_status(row[0])
            print "4: recorded={} and id ={} event ={} {}".format(tweet.created_at,row[0],current,datetime.datetime.now())
            df = df.append({'event_id': row[1],'tweet_id': row[0], 'user_id': tweet.user.id_str, 'screen_name': tweet.user.screen_name.encode("utf-8").strip(), 'signup_time': tweet.user.created_at, 'followers_count':tweet.user.followers_count,
         'friends_count': tweet.user.friends_count, 'location':tweet.user.location, 'verified' :tweet.user.verified, 
         'user_description':tweet.user.description.encode("utf-8"), 'listed_count': tweet.user.listed_count, 'favourites_count':tweet.user.favourites_count,
         'statuses_count': tweet.user.statuses_count, 'default_profile':tweet.user.default_profile, 'listed_count': tweet.user.listed_count,
         'tweet_created_at':tweet.created_at, 'tweet_content_text':tweet.text.encode("utf-8").strip(), 'retweet_count':tweet.retweet_count,
         'in_reply_to_status_id_str':tweet.in_reply_to_status_id_str, 'in_reply_to_user_id':tweet.in_reply_to_user_id,
         'favorite_count':tweet.favorite_count, 'entities':tweet.entities}, ignore_index=True)
            if  (tweet.created_at - date).total_seconds() > 36000 :
                current = current + 1
                get_last = True
                print "5: wrote to csv={} and id ={} event ={} {}".format(tweet.created_at,row[0],current,datetime.datetime.now())
                df.to_csv("output/hour10-events_{}.csv".format(in_file),encoding='utf-8')
                df.to_csv("output/backup/hour10-{}_{}.csv".format(index,in_file),encoding='utf-8')
        except Exception , e:
                    print(e)
                    print("a")
                    continue

1: current=829 and row =829 2019-12-27 13:59:29.257607
2: origin time=None id=410378929682546688 2019-12-27 13:59:29.258063
3: last time=2013-12-10 12:02:15 and id =410378929682546688 2019-12-27 13:59:32.068833
4: recorded=2013-12-10 12:02:15 and id =410378929682546688 event =829 2019-12-27 13:59:33.691127
1: current=829 and row =829 2019-12-27 13:59:33.724007
4: recorded=2013-12-10 12:02:15 and id =410378929308835840 event =829 2019-12-27 13:59:35.026693
1: current=829 and row =829 2019-12-27 13:59:35.061905
4: recorded=2013-12-10 12:02:15 and id =410378929133076480 event =829 2019-12-27 13:59:36.364557
1: current=829 and row =829 2019-12-27 13:59:36.388454
4: recorded=2013-12-10 12:02:15 and id =410378928784936960 event =829 2019-12-27 13:59:37.791906
1: current=829 and row =829 2019-12-27 13:59:37.809590
[{u'message': u'No status found with that ID.', u'code': 144}]
a
1: current=829 and row =829 2019-12-27 13:59:39.205287
4: recorded=2013-12-10 12:02:16 and id =410378933700685824 ev

KeyboardInterrupt: 

In [ ]:
df.to_csv("orginal/hour10-events{}.csv".format(in_file),encoding='utf-8')
df2.to_csv("orginal/lasttime_{}.csv".format(in_file),encoding='utf-8')
print "wrote to orginal {}".format(data.iloc[i][1])